In [123]:
# Load packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)


from torch.autograd import Variable
import os
import numpy as np



# input_dim = 5
# hidden_dim = 10
# n_layers = 1

# batch_size = 1
# seq_len = 3



In [19]:
inp = torch.randn(batch_size, seq_len, input_dim)
lstm_layer = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
cell_state = torch.randn(n_layers, batch_size, hidden_dim)
hidden = (hidden_state, cell_state)
out, hidden = lstm_layer(inp, hidden)
print("Output shape: ", out.shape)
print("Hidden: ", hidden)

In [143]:
trn_tokens, dev_tokens, max_length = load_data(os.getcwd()+"/data-for-lang-modeling/")
vocabulary = buildVocabulary(trn_tokens)
# trn_x, dev_x = index_tokens(trn_tokens, vocabulary), index_tokens(dev_tokens, vocabulary)
# padding(trn_x, max_length)
# len(vocabulary)
print(trn_tokens[-56:])

Training data ...
17556
Development data ...
1841
[['<start>', 'the', 'starling', 'family', 'sturnidae', 'is', 'an', 'entirely', 'old', 'world', 'group', 'apart', 'from', 'introductions', 'elsewhere', 'with', 'the', 'greatest', 'numbers', 'of', 'species', 'in', 'southeast', 'asia', 'and', 'sub', '@-@', 'saharan', 'africa', 'the', 'genus', 'sturnus', 'is', '<unk>', 'and', 'relationships', 'between', 'its', 'members', 'are', 'not', 'fully', 'resolved', 'the', 'closest', 'relation', 'of', 'the', 'common', 'starling', 'is', 'the', 'spotless', 'starling', 'the', 'non', '@-@', 'migratory', 'spotless', 'starling', 'may', 'be', 'descended', 'from', 'a', 'population', 'of', 'ancestral', 's.', 'vulgaris', 'that', 'survived', 'in', 'an', 'iberian', '<unk>', 'during', 'an', 'ice', 'age', 'retreat', 'and', 'mitochondrial', 'gene', 'studies', 'suggest', 'that', 'it', 'could', 'be', 'considered', 'as', 'a', 'subspecies', 'of', 'the', 'common', 'starling', 'there', 'is', 'more', 'genetic', 'variation'

In [141]:
# model = LSTMLanguageModel(len(vocabulary))
# print(model)
# res = model(torch.tensor(trn_x[1]))
# res[0].max()

# prepare_sequence(trn_tokens[1], vocabulary)
a = [1,2,3]
a[1:]

[2, 3]

In [136]:
# train the model
model = LSTMLanguageModel(len(vocabulary))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

with torch.no_grad():
    inputs = torch.tensor(trn_x[0])
    word_scores = model(inputs)
    print(word_scores.size())
    
i = 0
    
for epoch in range(3):
    for sentence in trn_x[:1000]:
        # clear out gradients
        model.zero_grad()
        
        # get inputs 
        sentence_in = torch.tensor(sentence)
        targets = torch.tensor(sentence)
        
        # run for word scores
        word_scores = model(sentence_in)
        
        # compute loss, gradients, update parameters
        loss = loss_function(word_scores, targets)
        if i%500==0: print(loss.item())
        loss.backward()
        optimizer.step()
        i += 1

# with torch.no_grad():
#     inputs = torch.tensor(trn_x[0])
#     word_scores = model(inputs)
#     print(word_scores.size())

torch.Size([111, 27767])
10.237525939941406


KeyboardInterrupt: 

In [137]:
def load_data(path):
    # Load data
    trn_texts = open(path+"trn-wiki.txt").read().strip().split("\n")
    dev_texts = open(path+"dev-wiki.txt").read().strip().split("\n")
    print("Training data ...")
    print("%d" % (len(trn_texts)))
    print("Development data ...")
    print("%d" % (len(dev_texts)))

    #tokenize data
    trn_tokens = [trn_text.split(" ") for trn_text in trn_texts]
    dev_tokens = [dev_text.split(" ") for dev_text in dev_texts]
    
    #get max feature size
    max_length = max(max([len(tokens) for tokens in trn_tokens]), max([len(tokens) for tokens in dev_tokens]))
    
    return trn_tokens, dev_tokens, max_length


def buildVocabulary(trn_tokens):
    words = set()
    vocab = {}
    index = 1
    for tokens in trn_tokens:
        for token in tokens:
            if token not in vocab:
                vocab[token] = index
                index += 1
    return vocab


def index_tokens(tokens, vocabulary):
    tokens_index_list = []
    for tokens in tokens_list:
        tokens_index = [vocabulary[token] for token in tokens]
        tokens_index_list.append(tokens_index)
    return tokens_index_list    

def prepare_sequence(tokens, vocabulary):
#     tokens_index_list = []
#     for tokens in tokens_list:
#         tokens_index = [vocabulary[token] for token in tokens]
#         tokens_index_list.append(tokens_index)
    tokens_index = [vocabulary[token] for token in tokens]
    return tokens_index    


# def padding(tokens_index_list, seq_len):
#     features = np.zeros((len(tokens_index_list), seq_len),dtype=int)
#     for i, tokens_index in enumerate(tokens_index_list):
#         if len(tokens_index) != 0:
#             features[i, -len(tokens_index):] = np.array(tokens_index)[:seq_len]
#     return features


# class LSTMLanguageModel(nn.Module):
#     def __init__(self, vocab_size, embedding_dim = 32, batch_size = 1, hidden_dim=32, num_layers=1):
#         super(LSTMLanguageModel, self).__init__()
#         self.batch_size = batch_size
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers
#         self.embedding_dim = embedding_dim
        
#         # Initialize a default nn.Embedding as word embeddings
#         self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
#         # LSTM layer
#         self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, num_layers = num_layers)
        
#         # Linear output layer
#         self.linear_output = nn.Linear(in_features=embedding_dim, out_features=vocab_size)
        
#     def init_weights(self):
#         init_range = 0.1
#         self.word_embeddings.weight.data.uniform_(-init_range, init_range)
#         self.linear_output.bias.data.fill_(0.0)
#         self.linear_output.weight.data.uniform_(-init_range, init_range)

#     def init_hidden(self):
#         weight = next(self.parameters()).data
#         return (Variable(weight.new(self.num_layers, self.batch_size, self.embedding_dim).zero_()),
#                 Variable(weight.new(self.num_layers, self.batch_size, self.embedding_dim).zero_()))
        
#     def forward(self, inputs, hidden):
#         embeds = self.word_embeddings(inputs)
#         lstm_out, hidden = self.lstm(embeds, hidden)
#         # flatted output
#         output = self.linear_output(lstm_out.view(-1, self.embedding_dim))
#         return output, hidden

    
class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim = 32, batch_size = 1, hidden_dim=32, num_layers=1):
        super(LSTMLanguageModel, self).__init__()
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_dim = embedding_dim
        
        # Initialize a default nn.Embedding as word embeddings
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, num_layers = num_layers)
        
        # Linear mid layer to transfer hidden to probablities
        self.linear = nn.Linear(in_features=embedding_dim, out_features=vocab_size)
        
#     def init_weights(self):
#         init_range = 0.1
#         self.word_embeddings.weight.data.uniform_(-init_range, init_range)
#         self.linear_output.bias.data.fill_(0.0)
#         self.linear_output.weight.data.uniform_(-init_range, init_range)

#     def init_hidden(self):
#         weight = next(self.parameters()).data
#         return (Variable(weight.new(self.num_layers, self.batch_size, self.embedding_dim).zero_()),
#                 Variable(weight.new(self.num_layers, self.batch_size, self.embedding_dim).zero_()))
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, hidden = self.lstm(embeds.view(len(sentence), self.batch_size, -1))
        word_space = self.linear(lstm_out.view(-1, self.embedding_dim))
        output = F.log_softmax(word_space, dim=1)
        return output